# Local Operator 

The finite element method itself is implemented in the so-called *local operator* realized by the class template
`PoissonP1`. It provides all the necessary element-local computations as described **in Subsection 3.4** and is declared as follows:

In [ ]:
template<typename F, typename FiniteElementMap>
class PoissonP1 :

  public Dune::PDELab::FullVolumePattern,
  public Dune::PDELab::LocalOperatorDefaultFlags
{

The first template parameter provides the right hand side function of the PDE and
the second parameter provides a finite element map giving access to finite element
basis functions on the reference element for all elements of the grid. The class derives
from the PDELab classes `FullVolumePattern` and `LocalOperatorDefaultFlags` which provide some default constants and methods.

The basic assumption of this implementation of the finite element method is that all elements of the mesh are simplices of dimension d which use the same polynomial degree 1. In order to make the code faster it is a good idea to do the evaluation of the basis functions and their gradients on the reference element once before the
computations start. This will be done in the constructor, but before we can do so we need to do some preparations.

## Type Definitions and Data Members

The class begins by extracting important types. The nite element map provides a finite element for each element of the map. Its type is

In [ ]:
private:
  // define useful types
  typedef typename FiniteElementMap::Traits::FiniteElementType
     FiniteElementType;

Among other things the finite element contains the basis functions on the reference element which can be accessed via the following type:

In [ ]:
  typedef typename FiniteElementType::Traits::LocalBasisType
     LocalBasisType;

DUNE thinks of basis functions on the reference element to be of
the most general form 
$$\hat\phi : \mathbb{A}^d \to \mathbb{B}^k, \quad 
\nabla\hat\phi : \mathbb{A}^d \to \mathbb{B}^{k\times d},$$
i.e. they may be vector-valued. The following type definitions

In [ ]:
  typedef typename LocalBasisType::Traits::DomainType
     DomainType;
  typedef typename LocalBasisType::Traits::RangeFieldType
     RF;
  typedef typename LocalBasisType::Traits::RangeType
     RangeType;
  typedef typename LocalBasisType::Traits::JacobianType
     JacobianType;

provide types to represent arguments and results of basis function
evaluations. `DomainType` represents $\mathbb{A}^d$,
`RF` represents $\mathbb{B}$, `RangeType`
represents $\mathbb{B}^k$ and finally `JacobianType`
represents $\mathbb{B}^{k\times d}$.

Next, we extract some important constants, the dimension 
of the grid and the number of basis functions per element:

In [ ]:
  // data members
  enum {dim=LocalBasisType::Traits::dimDomain};
  enum {n=dim+1};

As private data members the class stores an instance of the right hand side function `f` provided by the `driver`:

In [ ]:
  const F f;              // right hand side function

the midpoint quadrature rule

In [ ]:
  DomainType qp;          // center of mass of refelem
  double weight;          // quadrature weight on refelem

where `qp` is $\hat S_d$ and `weight` is $w_d$, and the values of the basis functions at the quadrature
point and their gradients:

In [ ]:
  double phihat[n];       // basis functions at qp
  double gradhat[dim][n]; // coordinate x #basisfct

Then, already in the public part, we need to define some constants that control the operation of the grid operator doing the global assembly:

In [ ]:
public:
  // define flags controlling global assembler
  enum { doPatternVolume = true };
  enum { doAlphaVolume = true };
  enum { doLambdaVolume = true };

These constants are evaluated at *compile time* and tell he grid operator class which methods have been implemented in the local operator by the user. Actually, the base class `LocalOperatorDefaultFlags` provides all possible flags ith the value`false` and we just need to overwrite the nes that are needed.
The constant`doPatternVolume` ells the global assembler to determine the sparsity pattern of the
matrix $A$ from a method `pattern_volume` which i inherited from the base class `FullVolumePattern`.
This default implementation inserts nonzeros between all degrees f freedom of an element. The constants `doAlphaVolume` and `doLambdaVolume` determine that our finite element method contains a volume integral involving the finite element solution $u_h$ and a right hand side integral which does not involve the finite element solution.

Setting `doAlphaVolume` to true implies that the local operator class implements the methods `alpha_volume`, 
`jacobian_apply_volume` and `jacobian_volume`.
Setting `doLambdaVolume` to true implies that the method `lambda_volume` must be implemented.

## Constructor

The constructor of the class has the following signature:

In [ ]:
  // Constructor precomputes element independent data
  PoissonP1 (const F& f_, const FiniteElementType& fel)
    : f(f_)

It takes the right hand side function `f` and a finite element `fel` as argument. The finite element is obtained from the finite element map and the first element of the grid in the function `driver`.

First thing to do is to get the lowest order quadrature rule for simplices from DUNE and we check that this is actually the midpoint rule:

In [ ]:
  {
    // select quadrature rule
    Dune::GeometryType gt = fel.type();
    const Dune::QuadratureRule<RF,dim>&
      rule = Dune::QuadratureRules<RF,dim>::rule(gt,1);
    if (rule.size()>1) {
      std::cout << "Wrong quadrature rule!" << std::endl;
      exit(1);
    }

Then we store the first quadrature point in the local data members:

In [ ]:
    // position and weight of the quadrature point
    weight = rule[0].weight();
    qp = rule[0].position();

It is also a good idea to check that the basis given by the user has at least the correct size:

In [ ]:
    // check size of the basis
    if (fel.localBasis().size()!=n) {
      std::cout << "Wrong basis!" << std::endl;
      exit(1);
    }

Now the basis functions can be evaluated at the quadrature point in the reference element and the results are stored in the data members of the class:

In [ ]:
    // evaluate basis functions on refelem
    std::vector<RangeType> phi(n);
    fel.localBasis().evaluateFunction(qp,phi);
    for (int i=0; i<n; i++) phihat[i] = phi[i];

And the same now for the gradients:

In [ ]:
    // evaluate gradients of basis functions on refelem
    std::vector<JacobianType> js(n);
    fel.localBasis().evaluateJacobian(qp,js);
    for (int i=0; i<n; i++)
      for (int j=0; j<dim; j++)
        gradhat[j][i] = js[i][0][j];
  }

Note that the last index loops over the number of basis functions. 

## Method `lambda_volume`

This method computes the contributions $b_T$ to the right hand side vector for a given element as given in Eq.XXX.
It has the following signature:

In [ ]:
  // volume integral depending only on test functions
  template<typename EG, typename LFSV, typename R>
  void lambda_volume (const EG& eg, const LFSV& lfsv,
                      R& r) const

Argument`eg` provides the element $T$ in a wrapped form such that PDELab need not operate directly on a DUNE grid.
With `eg.geometry()` the geometry of the element can be accessed in the form of a `Dune::Geometry`. With `eg.entity()` one can access the underlying codim 0 entity of the DUNE grid.
The second argument `lfsv` provides the test functions on the reference element and `r` provides a container where the result should be stored.


First thing to do is to evaluate the right hand side function at the quadrature point:

In [ ]:
  {
    typename F::Traits::RangeType fval;
    f.evaluate(eg.entity(),qp,fval);

Next, we compute the factor that is common to all entries of $b_T$:

In [ ]:
    RF factor=fval*weight*eg.geometry().integrationElement(qp);

Note that the method `integrationElement` on the geometry provides the value of $|\det B_T|$.

Finally, we can compute the entries and store them in the results container:

In [ ]:
    for (int i=0; i<n; i++)
      r.accumulate(lfsv,i,-factor*phihat[i]);
  }

Here it is important to note the minus sign because PDELab actually solves the weak formulation as 
$$r(u_h,v) = a(u_h,v)-l(v) = 0 \quad \forall v\in V$$ since this is more appropriate in the case of nonlinear partial differential equations.

## Method `jacobian_volume`

Next we need to compute the element contributions to the stiffness matrix as described in Eq.XXX.
This is done by the method `jacobian_volume` with

In [ ]:
  // jacobian of volume term
  template<typename EG, typename LFSU, typename X,
           typename LFSV, typename M>
  void jacobian_volume (const EG& eg, const LFSU& lfsu,
                        const X& x, const LFSV& lfsv,
                        M& mat) const
  {

Its arguments are: `eg` providing the wrapped codim 0 entity $T$,`lfsu` providing the basis functions of the ansatz space, `x` providing the coefficients of the current iterate of the finite element solution, `lfsv` providing the test functions and `mat` a container to store the result.

The `jacobian_volume` method works in the same way also for nonlinear problems. Nonlinear problems are solved iteratively, e.g. using Newton's method or a fixed-point iteration, where the method should provide a linearization at the current iterate given by the combination of `lfsu` and `x`.
In our case of a linear problem the result *does not depend* on the current iterate.
Moreover, the basis functions for the test space are precomputed so we need
not access them via `lfsv`. Note also that in general the ansatz and test space might be different.

First thing we need is to get $B_T^{-T}$ and store it into `S`:

In [ ]:
    // get Jacobian and determinant
    // assume the transformation is linear
    const auto geo = eg.geometry();
    const auto S = geo.jacobianInverseTransposed(qp);

Next, $|\det B_T|$ is retrieved from the geometry and the factor that is common to all entries of the local stiffness matrix is computed:

In [ ]:
    RF factor = weight*geo.integrationElement(qp);

Now form the matrix of transformed gradients $G=B_T^{-T} \hat G$ and store it in `grad`:

In [ ]:
    // compute gradients of basis functions in transformed element
    double grad[dim][n] = {{0.0}}; // coordinate x #basisfct
    for (int i=0; i<dim; i++) // rows of S
      for (int k=0; k<dim; k++) // columns of S
        for (int j=0; j<n; j++) // columns of gradhat
          grad[i][j] += S[i][k] * gradhat[k][j];

The computations are arranged in such a way that the innermost loop has the dimension number of basis functions. In 3d there are four basis functions and the loop has a chance to get vectorized.

Now the local stiffness matrix $A_T = G^T G$ (up to the factor $|\det B_T| w_d$) is formed

In [ ]:
    // compute grad^T * grad
    double A[n][n] = {{0.0}};
    for (int i=0; i<n; i++)
      for (int k=0; k<dim; k++)
        for (int j=0; j<n; j++)
          A[i][j] += grad[k][i]*grad[k][j];

and stored in the results container (now multiplying with the common factor):

In [ ]:
    // store in result
    for (int i=0; i<n; i++)
      for (int j=0; j<n; j++)
        mat.accumulate(lfsu,i,lfsu,j,A[i][j]*factor);
  }

## Method `alpha_volume`

The method `alpha_volume` provides the element-local computations for the matrix-free evaluation of $a(u_h,\phi_i)$ for all test functions $\phi_i$ as given by Eq.XXX. It has the interface:

In [ ]:
 // volume integral depending on test and ansatz functions
  template<typename EG, typename LFSU, typename X,
           typename LFSV, typename R>
  void alpha_volume (const EG& eg, const LFSU& lfsu,
                     const X& x, const LFSV& lfsv,
                     R& r) const

Its arguments are: `eg` providing the wrapped codim 0 entity $T$,`lfsu` providing the basis functions of the ansatz space,`x` providing the coefficients of the current iterate of the finite element solution, `lfsv` providing the test functions and `r` a container to store the result.

The computations are actually quite similar to those in `jacobian_volume`. In particular, the computation of $B_T^{-T}$, $|\det B_T|$ and $G = B_T^{-T} \hat G$ are the same.

In [ ]:
  {
    // get Jacobian and determinant
    // assume the transformation is linear
    const auto geo = eg.geometry();
    const auto S = geo.jacobianInverseTransposed(qp);
    RF factor = weight*geo.integrationElement(qp);

    // compute gradients of basis functions in transformed element
    double grad[dim][n] = {{0.0}};  // coordinate x #basisfct
    for (int i=0; i<dim; i++) // rows of S
      for (int k=0; k<dim; k++) // columns of S
        for (int j=0; j<n; j++) // columns of gradhat
          grad[i][j] += S[i][k] * gradhat[k][j];

Extracting the element local coefficients $z_T = R_T z$ is done by:

In [ ]:
    double z_T[n];
    for (int j=0; j<n; j++) z_T[j] = x(lfsu,j); // read coeffs

Now we may compute $\nabla u_h$ via $G z_T$:

In [ ]:
    double graduh[dim] = {0.0};
    for (int k=0; k<dim; k++) // rows of grad
      for (int j=0; j<n; j++) // columns of grad
        graduh[k] += grad[k][j]*z_T[j];

Finally, the result $a_T = G^T \nabla u_h$ is formed:

In [ ]:
    // scalar products
    double a_T[n] = {0.0};
    for (int k=0; k<dim; k++) // rows of grad
      for (int j=0; j<n; j++)
        a_T[j] += grad[k][j]*graduh[k];


and stored in the results container (while being multiplied with the common factor):

In [ ]:
    // store in result
    for (int i=0; i<n; i++)
      r.accumulate(lfsv,i,a_T[i]*factor);
  }

## Method `jacobian_apply_volume`

In the case of a *nonlinear* partial differential equation the finite element method results in a weak form 
$$u_h \in V_h \ : \quad r(u_h,v) = \alpha(u_h,v) - \lambda(v) = 0 \quad \forall v\in V_h$$
which is *nonlinear* in its *first* argument. Inserting the finite element basis results in a nonlinear algebraic problem
$$R(z) = 0$$
with $(R(z))_i = r(\text{FE}_h(z),\phi_i)$ which is typically solved by Newton's iteration
or some other iterative method. In case of Newton's method, each step
involves the solution of a {\em linear} system of the form
$$J(z)\, w = R(z)$$
where $(J(z))_{i,j} = \frac{(\partial R(z))_i}{\partial z_j} = 
\frac{\partial \alpha(\text{FE}_h(z),\phi_i)}{\partial z_j}$ is the Jacobian of 
the nonlinear map $R$.

Naturally, the nonlinear case also includes the linear case described in this tutorial
by setting $r(u,v) = a(u,v)-l(v)$. Then, due to the linearity of $a$ in its first
argument, one can show that $J(z) = A$ and 
$$(J(z)\, w)_i = (Aw)_i = a(\text{FE}_h(w),\phi_i).$$
This is *not* true in the nonlinear case. There, the evaluation of the form $\alpha(\text{FE}_h(w),\phi_i)$ and the application of the Jacobian $J w$ are different operations. Therefore, PDELab provides two functions with
the application of the Jacobian implemented in `jacobian_apply_volume`
with the interface:

In [ ]:
  //! apply local jacobian of the volume term
  template<typename EG, typename LFSU, typename X,
           typename LFSV, typename R>
  void jacobian_apply_volume (const EG& eg, const LFSU& lfsu,
                              const X& z, const LFSV& lfsv,
                              R& r) const

Note, this is the same interface as for `alpha_volume`. Since our problem is linear, Jacobian application is identical to bilinear form evaluation and therefore we may just forward the call to the
function `alpha_volume`:

In [ ]:
  {
    alpha_volume(eg,lfsu,z,lfsv,r);
  }

END Tut00.pdf, what about his second function for jacobian_apll_vol?

In [ ]:



  //! apply local jacobian of the volume term
  template<typename EG, typename LFSU, typename X,
           typename LFSV, typename R>
  void jacobian_apply_volume (const EG& eg, const LFSU& lfsu,
                              const X& x, const X& z, const LFSV& lfsv,
                              R& r) const
  {
    alpha_volume(eg,lfsu,z,lfsv,r);
  }
};
%%%block-end